In [1]:
!pip install -q h5py pyyaml

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

import os
import tensorflow as tf
from tensorflow import keras


In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 1s 0us/step


In [3]:
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model


# 모델 객체를 만듭니다
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # 훈련 단계에 콜백을 전달합니다

W0702 10:59:32.526869 12084 deprecation.py:323] From C:\Users\Magenta\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 864/1000 [========================>.....] - ETA: 0s - loss: 1.2322 - accuracy: 0.6586
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 292us/sample - loss: 1.1347 - accuracy: 0.6850 - val_loss: 0.7494 - val_accuracy: 0.7640
Epoch 2/10
 864/1000 [========================>.....] - ETA: 0s - loss: 0.4155 - accuracy: 0.8877
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 121us/sample - loss: 0.4236 - accuracy: 0.8840 - val_loss: 0.5166 - val_accuracy: 0.8350
Epoch 3/10
 864/1000 [========================>.....] - ETA: 0s - loss: 0.2722 - accuracy: 0.9398
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 116us/sample - loss: 0.2735 - accuracy: 0.9390 - val_loss: 0.4655 - val_accuracy: 0.8590
Epoch 4/10
 864/1000 [========================>.....] - ETA: 0s - loss: 0.1993 - accuracy: 0.9537
Epoch

In [7]:
model = create_model()
loss, acc = model.evaluate(test_images, test_labels)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))


1000/1000 [==============================] - 0s 55us/sample - loss: 2.3922 - accuracy: 0.0780
훈련되지 않은 모델의 정확도:  7.80%


In [8]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 21us/sample - loss: 0.4298 - accuracy: 0.8640
복원된 모델의 정확도: 86.40%


In [9]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # 다섯 번째 에포크마다 가중치를 저장합니다
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)

W0702 11:05:34.645651 12084 callbacks.py:859] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.



Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

W0702 11:06:44.070183 12084 util.py:244] Unresolved object in checkpoint: (root).optimizer.iter
W0702 11:06:44.071180 12084 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_1
W0702 11:06:44.072177 12084 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_2
W0702 11:06:44.072177 12084 util.py:244] Unresolved object in checkpoint: (root).optimizer.decay
W0702 11:06:44.072177 12084 util.py:244] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0702 11:06:44.073176 12084 util.py:252] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/alpha/guide/checkpoints#loading_mechanics for details.


1000/1000 [==============================] - 0s 55us/sample - loss: 0.4946 - accuracy: 0.8750
복원된 모델의 정확도: 87.50%
